<a href="https://colab.research.google.com/github/WhiteHum/Application-security/blob/main/4_01_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

## Overview

In this lab you will leverage some of your existing skills to extract NetFlow data from the repository on the VM, plot that data, and then attempt to fit that data using several different regression methods.

## Goals

By the end of this lab you should be able to:

 * Better extract remote data and import it into a Python process
 * Perform a Linear Regression
 * Perform regressions using higher order functions
 * Plot and analyze the outputs of your regressions
 * Measure the accuracy of these regressions
 
## Estimated Time: 45 - 60 minutes

# Obtaining the Data
This lab will leverage your existing knowledge of numpy, matplotlib, and the SSH class that we created on Day 2.

Our first goal is to successfully read NetFlow statistics that tell us the number of bytes per hour that pass over the network.  Once we have this data, we want to graph it.  Once we know what the data looks like and are satisfied that we have imported it correctly, we will create a forecasting function using regression analysis.  To attempt to measure how well our forecasting function works, we will only use a subset of the ground-truth data for the regression.  This will allow us to measure how well the function forecasts other known data.

# <img src="../images/task.png" width=20 height=20> Task 1.1

Please use the following cell to:
 * Import the *numpy* library with the typical `np` alias
 * Import the *pyplot* package from *matplotlib* using the typical `plt` alias
 * Recreate (or copy) the SSH class that we developed in the final lab during Day 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ssh2.session import Session
import socket

class SSHCommand:
    address = ""
    username = ""
    password = ""
    
    def __init__(self, address="127.0.0.1", username="none", password="none"):
        self.address = address
        self.username = username
        self.password = password
    
    def execute(self, command = ""):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((self.address, 22))
        session = Session()
        session.handshake(sock)
        session.userauth_password(self.username, self.password)
        channel = session.open_session()
        channel.execute(command)
        result = ""
        size, result = channel.read()
        while size > 0:
            size, data = channel.read()
            result = result + data
        channel.close()
        session.disconnect()
        return result.decode('ascii')


Assuming that our libraries are loaded and our SSH class is ready to be used, let's just test things out.

# <img src="../images/task.png" width=20 height=20> Task 1.2

In the following cell, use the SSH class to execute the following command against the NetFlow repository.  Capture the result in a variable named `data`.

**Remote Command to Execute:**
```
rwfilter --type=all --start-date=2020/01/01 --end-date=2021/03/01 --proto=0-255 --pass=stdout | rwcount --no-titles --delimited=, --bin-size=86400
```

# <img src="../images/task.png" width=20 height=20> Task 1.3

Before continuing, we should make sure that `data` contains what we think it should.  This allows us to verify that our command has executed correctly and the data has been returned.  This also allows us to inspect the structure of the data so that we can prepare to parse it.

Print the value of `data` that was returned.  Verify that it appears to be flow data giving the statistics over a period of 1 day periods from around January 1, 2020 through February 20, 2021.

Now that our data has been returned we can get to work preprocessing it.  What we appear to have are a series of comma separated fields on lines with the following structure:

|     Date           | Records/Flows | Bytes      | Packets         |
|:------------------:|:-------------:|:----------:|:---------------:|
|2019/05/03T12:55:00|1231.73|11576472.78|36115.26|
|2019/05/03T13:00:00|5100.00|28034694.10|67292.59|
|2019/05/03T13:05:00|4190.39|19801249.46|52100.03|
|2019/05/03T13:10:00|11359.62|9700616.61|38636.73|

What we need to do is turn these into a numpy array.  Before we do, let's think about what it is we're trying to do and whether there is any data that we can drop.  Right now we are working on producing a graph representing the amount of data (bytes).  Could we easily imagine wanting to see a graph of the number of flows or the number of packets?  Certainly.  Given this, we probably want to pull out that data as well.

On the other hand, does the timestamp matter so much?  If we are trying to create a dashboard that shows the amount of data at a specific time, then yes.  However, that's not our task.  What we're trying to do is look at the data over time in one day increments.  The actual time or date doesn't matter much at this point, and may never matter.  The only thing that is relevant is that the data is presented in the correct order.

Given these facts, we would like to keep the last three fields on each line and discard the first.

# Convert and Visualize the Data

# <img src="../images/task.png" width=20 height=20> Task 1.4

Write Python code that extracts the number of flows, bytes, and packets from each line of data, aggregating these lines into a numpy array named `stats` that contains data of type `np.float32`.  To help you verify success, `stats` should have the shame `(331,3)` when you are done.

# <img src="../images/task.png" width=20 height=20> Task 1.5

Now that we have turned the data into a numpy array, we are ready to have a look at the data.  Please write a Python function that will allow you to pass in a set of values in a numpy array.  The function should use matplotlib to draw a visualization of this data.

Once this function is written, generate plots of the flows, bytes, and packets present in the `stats` array.